# Agentic Cooking Intelligence — Dual-Mode (Offline/Online) — **Course Edition**

This notebook includes the same topics that were presented in the scene parsing master class; because of the complexity of the demo code, I built you this demo notebook so that you could more easily explore the underlying agentic concepts.  Thanks so much for being part of our program.  (Will)

## 1) Mode & Proxy Setup

In [1]:
# === Runtime Mode ===
MODE = "ONLINE"  # change to "OFFLINE" for no-network runs

# Provide your GL key via env/secret or paste here (for ONLINE)
from google.colab import userdata

try:
    GL_OpenAI = userdata.get('GL_OpenAI') # noqa
except NameError:
    GL_OpenAI = None  # e.g., "gl-..."

API_BASE = "https://aibe.mygreatlearning.com/openai/v1"
MODEL_ID = "gpt-4o-mini"
EMBED_MODEL = "text-embedding-3-small"

def is_online():
    return MODE.upper() == "ONLINE" and GL_OpenAI not in (None, "", "REPLACE_ME")

print("MODE:", MODE, "| GL key configured:", bool(GL_OpenAI and GL_OpenAI != "REPLACE_ME"), "| Online:", is_online())

MODE: ONLINE | GL key configured: True | Online: True


## 2) 📦 UI Helpers (Pretty Output)

In [2]:
from IPython.display import display, HTML
import json as _json, html as _html
import pandas as pd
import textwrap

def show_card(title, subtitle=None, body_html=""):
    subtitle = subtitle or ""
    styles = (
        "<style>"
        ".nb-card{border:1px solid #e5e7eb;border-radius:14px;padding:16px;margin:10px 0;background:#fff;"
        "box-shadow:0 1px 2px rgba(0,0,0,0.04);}"
        ".nb-title{font-weight:700;font-size:18px;margin-bottom:6px;}"
        ".nb-sub{color:#6b7280;font-size:13px;margin-bottom:10px;}"
        ".nb-body{font-size:14px;line-height:1.45}"
        ".nb-badge{display:inline-block;padding:2px 8px;border-radius:9999px;border:1px solid #e5e7eb;"
        "font-size:12px;margin-left:6px;}"
        "</style>"
    )
    html = (
        styles +
        '<div class="nb-card">'
        f'<div class="nb-title">{title}</div>'
        f'<div class="nb-sub">{subtitle}</div>'
        f'<div class="nb-body">{body_html}</div>'
        "</div>"
    )
    display(HTML(html))

def show_json(obj, title="JSON"):
    formatted = _json.dumps(obj, indent=2, ensure_ascii=False)
    body = '<pre style="white-space:pre-wrap">' + _html.escape(formatted) + '</pre>'
    show_card(title, body_html=body)

def show_table(rows, columns=None, title="Table", subtitle=None, index=False):
    df = pd.DataFrame(rows, columns=columns) if columns else pd.DataFrame(rows)
    show_card(title, subtitle=subtitle, body_html=df.to_html(index=index))

def show_hits(hits, title="Retrieved Context"):
    rows = [{"#": i+1, "key": k, "score": round(s, 3)} for i,(k,_,s) in enumerate(hits)]
    show_table(rows, title=title, subtitle=f"{len(rows)} items", index=False)

def show_coverage(cov, title="Coverage"):
    found = ", ".join(cov.get("found", [])) or "—"
    missing = ", ".join(cov.get("missing", [])) or "—"
    pct = f"{int(100*cov.get('coverage',0))}%"
    body = "<b>Coverage:</b> " + pct + "<br/><b>Found:</b> " + found + "<br/><b>Missing:</b> " + missing
    show_card(title, body_html=body)

def show_badge(label, ok, note=""):
    color = "#10b981" if ok else "#f59e0b"
    text = "PASS" if ok else "SKIP/FAIL"
    html = (
        '<div class="nb-card">'
        f'<div class="nb-title">{label}<span class="nb-badge" style="border-color:{color};color:{color};">{text}</span></div>'
        f'<div class="nb-body">{textwrap.fill(note, 120)}</div>'
        "</div>"
    )
    display(HTML(html))

## 3) Clients — GL Chat/Embeddings (online) + FakeLM (offline)

In [3]:
import json, requests, numpy as np

class GLChatClient:
    def __init__(self, api_key, api_base=API_BASE, model=MODEL_ID):
        self.api_key = api_key; self.api_base = api_base.rstrip("/"); self.model=model
    def generate(self, messages, **kwargs):
        url = f"{self.api_base}/chat/completions"
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
        payload = {"model": self.model, "messages": messages}
        payload.update(kwargs or {})
        r = requests.post(url, headers=headers, json=payload, timeout=60)
        r.raise_for_status(); data = r.json()
        return data["choices"][0]["message"]["content"]

class GLEmbedClient:
    def __init__(self, api_key, api_base=API_BASE, model=EMBED_MODEL):
        self.api_key = api_key; self.api_base = api_base.rstrip("/"); self.model=model
    def embed(self, texts):
        if isinstance(texts, str): texts=[texts]
        url = f"{self.api_base}/embeddings"
        headers = {"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
        payload = {"model": self.model, "input": texts}
        r = requests.post(url, headers=headers, json=payload, timeout=60)
        r.raise_for_status(); data = r.json()
        vecs = [np.array(d["embedding"], dtype=np.float32) for d in data["data"]]
        return np.vstack(vecs)

class FakeLM:
    def generate(self, messages, **kwargs):
        last = ""
        for m in messages[::-1]:
            c = m.get("content")
            if isinstance(c, str): last = c; break
        tips = []
        l = last.lower()
        if "emulsif" in l: tips += ["Reserve starchy water and fat to emulsify.", "Finish pasta in pan with vigorous tossing."]
        if "gluten" in l:  tips += ["Use rice/corn GF pasta; avoid cross-contamination."]
        if "safety" in l or "allergen" in l: tips += ["Mind the 5–60°C danger zone.", "Ask for allergies; swap ingredients safely."]
        if not tips: tips = ["Mise en place.", "Taste & adjust.", "Control heat and time."]
        return "\n".join("- " + t for t in tips[:6])

chat_client = GLChatClient(GL_OpenAI) if is_online() else FakeLM()

def chat(system, user, **kwargs):
    messages = []
    if system: messages.append({"role":"system","content":system})
    messages.append({"role":"user","content":user})
    try:
        return chat_client.generate(messages, **kwargs)
    except Exception as e:
        return "[chat error] " + str(e)

## 4) Tiny In-Notebook Corpora

In [4]:
from collections import Counter
import math

task_corpus = {
    "pasta_basics.md": "Boil 1–2% salted water (~1L/100g). Reserve starchy water for emulsifying sauces.",
    "tomato_sauce.md": "Sauté garlic in oil, add tomatoes, simmer to thicken. Emulsify with pasta water; finish with basil.",
    "knife_safety.md": "Sharp knife, claw grip, stable board. Keep fingers tucked; cut away from yourself.",
}
policy_corpus = {
    "allergens.md": "Common allergens: gluten, dairy, nuts, eggs. Ask for allergies; suggest alternatives.",
    "food_safety.md": "Danger zone 5–60°C. Reheat leftovers to 74°C; refrigerate within 2 hours.",
    "substitutions.md": "If missing basil, try parsley/oregano. Gluten-free pasta alternatives: corn/rice-based.",
}
cuisine_corpora = {
    "eurocentric": [
        ("italian_pasta", "Pasta al pomodoro with olive oil, garlic, basil, tomatoes; emulsify with starchy water."),
        ("french_omelette", "Butter, eggs, fine herbs; avoid browning for tenderness."),
        ("greek_salad", "Tomatoes, cucumbers, olives, feta, olive oil, oregano; typically no lettuce."),
    ],
    "global": [
        ("italian_pasta", "Pasta al pomodoro..."),
        ("mexican_tacos", "Al pastor uses achiote, pineapple, pork on a trompo; serve on corn tortillas."),
        ("indian_dal", "Dal tadka with tempered cumin, mustard seed, garlic, chili."),
        ("japanese_miso", "Miso soup with dashi; add tofu, wakame, scallions off heat."),
        ("thai_green_curry", "Green curry paste + coconut milk; Thai basil, kaffir lime, fish sauce, palm sugar."),
    ],
    "mexican": [
        ("tacos_al_pastor", "Achiote-marinated pork with pineapple on a trompo; thin slices on corn tortillas."),
        ("salsa_verde", "Tomatillos, jalapeño, onion, cilantro, lime; char or boil tomatillos before blending."),
        ("frijoles", "Pinto beans simmered with onion/epazote; mash with cooking liquid for refritos."),
    ],
    "indian": [
        ("dal_tadka", "Toor/moong dal; temper cumin, mustard, garlic, chilies in ghee; finish with cilantro."),
        ("chana_masala", "Chickpeas with onion, tomato, ginger-garlic, chole masala, amchur."),
        ("jeera_rice", "Basmati with cumin tempered in ghee; rinse well for separate grains."),
    ],
    "japanese": [
        ("miso_soup", "Dashi with miso; add tofu, wakame, scallions off heat to avoid boiling miso."),
        ("tamago_yaki", "Rolled omelette with dashi/sugar; cook in layers in rectangular pan."),
        ("onigiri", "Salted rice balls; fillings like umeboshi or salmon; wrap with nori."),
    ],
    "italian": [
        ("pomodoro", "Tomatoes, garlic, olive oil, basil; finish pasta in pan with starchy water."),
        ("cacio_e_pepe", "Pecorino + black pepper; create a creamy emulsion with starchy water."),
        ("pesto_genovese", "Basil, pine nuts, garlic, olive oil, Parmigiano; keep cool to reduce oxidation."),
    ],
}
cuisine_keywords = {
    "mexican": {"corn","tortillas","achiote","tomatillo","cilantro","jalapeño","epazote","lime","beans"},
    "indian": {"dal","cumin","mustard","ghee","garam","chana","turmeric","masala","basmati","tadka"},
    "japanese": {"dashi","miso","tofu","wakame","scallions","nori","umeboshi","tamago","katsuobushi"},
    "italian": {"olive","basil","tomatoes","pecorino","parmigiano","garlic","pasta","emulsion","starchy"},
}

## 5) Retrieval Helpers (BM25-lite)

In [5]:
def _bm25lite_score(query, doc):
    q_terms = query.lower().split(); d_terms = doc.lower().split()
    df = Counter(d_terms); score = 0.0
    for t in q_terms:
        tf = df.get(t, 0); score += tf / (tf + 0.5)
    score += len(set(q_terms) & set(d_terms)) * 0.1
    return score

def retrieve_task(query, k=3):
    scored = sorted((( _bm25lite_score(query, v), k, v) for k,v in task_corpus.items()), reverse=True)[:k]
    return [(name, text, score) for (score, name, text) in scored]

## 6) Cuisine Vector Store — GL Embeddings **or** TF-IDF (auto)

In [6]:
def cosine_sim(a, b):
    a = a / (np.linalg.norm(a, axis=-1, keepdims=True) + 1e-9)
    b = b / (np.linalg.norm(b, axis=-1, keepdims=True) + 1e-9)
    return a @ b.T

def build_cuisine_index(docs):
    if is_online():
        try:
            embedder = GLEmbedClient(GL_OpenAI)
            texts = [t for _, t in docs]
            mat = embedder.embed(texts)
            keys = [k for k,_ in docs]
            return ("embed", {"mat": mat, "keys": keys, "docs": docs})
        except Exception as e:
            print("[Embedding path error -> TF-IDF fallback]", e)
    # TF-IDF fallback
    vocab, df, tokenized = {}, Counter(), []
    for k,v in docs:
        toks=v.lower().split(); tokenized.append((k,toks))
        for t in set(toks): df[t]+=1; vocab.setdefault(t,len(vocab))
    N=len(docs); import math
    idf={t: math.log((1+N)/(1+df[t]))+1 for t in df}
    mat, keys = [], []
    for k,toks in tokenized:
        tf=Counter(toks); import numpy as np
        vec=np.zeros(len(vocab), dtype=np.float32)
        for t,c in tf.items():
            j=vocab.get(t)
            if j is not None: vec[j]=(c/len(toks))*idf.get(t,1.0)
        nrm=np.linalg.norm(vec) or 1.0
        mat.append(vec/nrm); keys.append(k)
    import numpy as np
    return ("tfidf", {"vocab":vocab,"idf":idf,"mat":np.vstack(mat),"keys":keys,"docs":docs})

def search_cuisine(index, query, top_k=3):
    mode, state = index[0], index[1]
    # Primary: embedding path if available
    if mode == "embed" and is_online():
        try:
            qvec = GLEmbedClient(GL_OpenAI).embed([query])
            sims = cosine_sim(qvec, state["mat"]).ravel()
            order = np.argsort(-sims)[:top_k]
            return [(state["docs"][i][0], state["docs"][i][1], float(sims[i])) for i in order]
        except Exception as e:
            print("[Search embedding error -> TF-IDF fallback]", e)
    # Robust fallback: build a temporary TF-IDF index from docs in state if needed
    docs = state.get("docs") if isinstance(state, dict) else None
    if docs is None:
        # If the structure is already TF-IDF, proceed; otherwise try to recover docs list
        docs = state.get("docs", [])
    # If we already have TF-IDF keys, use them directly
    if all(k in state for k in ("vocab","idf","mat","keys","docs")):
        st = state
    else:
        # Build a minimal TF-IDF index on the fly
        from collections import Counter as _Counter
        import numpy as _np, math as _math
        vocab, df, tokenized = {}, _Counter(), []
        for k,v in docs:
            toks=v.lower().split(); tokenized.append((k,toks))
            for t in set(toks): df[t]+=1; vocab.setdefault(t,len(vocab))
        N=len(docs) or 1
        idf={t: _math.log((1+N)/(1+df[t]))+1 for t in df}
        mat, keys = [], []
        for k,toks in tokenized:
            tf=_Counter(toks); vec=_np.zeros(len(vocab), dtype=_np.float32)
            for t,c in tf.items():
                j=vocab.get(t)
                if j is not None: vec[j]=(c/len(toks))*idf.get(t,1.0)
            nrm=_np.linalg.norm(vec) or 1.0
            mat.append(vec/nrm); keys.append(k)
        st = {"vocab":vocab,"idf":idf,"mat":_np.vstack(mat) if len(mat)>0 else _np.zeros((0,0), dtype=_np.float32),
              "keys":keys,"docs":docs}
    # TF-IDF scoring
    vocab,idf,mat,keys,docs = st["vocab"],st["idf"],st["mat"],st["keys"],st["docs"]
    import numpy as _np
    toks=query.lower().split()
    q=_np.zeros(mat.shape[1] if mat.size else len(vocab) or 1, dtype=_np.float32)
    from collections import Counter as _Counter2
    tf=_Counter2(toks)
    for t,c in tf.items():
        j=vocab.get(t)
        if j is not None:
            q[j]=(c/max(1,len(toks)))*idf.get(t,1.0)
    denom = _np.linalg.norm(q)
    if denom: q = q/denom
    sims = (mat @ q) if mat.size else _np.zeros((len(docs),), dtype=_np.float32)
    order = _np.argsort(-sims)[:min(top_k, len(docs))]
    return [(docs[i][0], docs[i][1], float(sims[i])) for i in order]

## Demo: Task Helper

In [7]:
def rag_task(query, k=3):
    hits = retrieve_task(query, k=k)
    context = "\n\n".join(["### " + n + "\n" + t for n,t,_ in hits])
    ans = chat("Answer only from the provided context.", "Context:\n" + context + "\n\nQ: " + query + "\nAnswer concisely.")
    return ans, hits

ans, hits = rag_task("How do I make tomato sauce cling to pasta?")
show_hits(hits, title="RAG — Retrieved")
show_card("RAG — Answer", body_html="<pre style='white-space:pre-wrap'>" + str(ans) + "</pre>")

#,key,score
1,tomato_sauce.md,0.767
2,pasta_basics.md,0.000
3,knife_safety.md,0.000




## 8) Demo: Safety & Guidance

In [8]:
def rag_policy(query, k=2):
    task_hits = retrieve_task(query, k=k)
    task_ctx = "\n\n".join(["### " + n + "\n" + t for n,t,_ in task_hits])
    policy_ctx = "\n\n".join(["### " + n + "\n" + t for n,t in policy_corpus.items()])
    context = task_ctx + "\n\n" + policy_ctx
    ans = chat("Use the context; include safety/allergen guidance as relevant.", "Context:\n" + context + "\n\nQ: " + query + "\nProvide steps + safety notes + substitutions.")
    return ans, task_hits

ans_b, hits_b = rag_policy("Make a quick tomato pasta for someone who might be gluten-sensitive.")
show_card("RAG — Answer", body_html="<pre style='white-space:pre-wrap'>" + str(ans_b) + "</pre>")

## 9) Demo: Cuisine Vector RAG + Coverage Check

In [9]:
def cuisine_coverage(answer, cuisine):
    keys = cuisine_keywords.get(cuisine.lower(), set())
    found = {k for k in keys if k in answer.lower()}
    return {"coverage": len(found)/max(1,len(keys)), "found": sorted(list(found)), "missing": sorted(list(keys - found))}

def rag_cuisine(corpus, query, cuisine_hint=None, top_k=3):
    docs = cuisine_corpora.get(corpus, [])
    idx = build_cuisine_index(docs)
    hits = search_cuisine(idx, query, top_k=top_k)
    context = "\n\n".join(["### " + k + "\n" + v for k,v,_ in hits])
    guard = "If context is culturally mismatched or insufficient, clearly note the gap and request clarification."
    ans = chat("You answer using only context. " + guard, "Context:\n" + context + "\n\nQ: " + query + "\nConcise, culturally aware answer.")
    cov = cuisine_coverage(ans, cuisine_hint or corpus)
    return ans, hits, cov

corpus, q, hint = "global", "Make gluten-free tacos with a bright salsa", "mexican"
a, h, c = rag_cuisine(corpus, q, cuisine_hint=hint, top_k=3)
show_hits(h, title="RAG — Hits (" + corpus + ")")
show_card("RAG — Answer", subtitle="query: " + q + " | hint: " + hint, body_html="<pre style='white-space:pre-wrap'>" + str(a) + "</pre>")
show_coverage(c, title="RAG — Coverage")

#,key,score
1,mexican_tacos,0.398
2,japanese_miso,0.320
3,indian_dal,0.294


## 10) Bias Surfacing and Mitigation

In [10]:
def rate_candidate(c):
    prompt = (
        "Rate suitability using **task-relevant skills only**. Avoid prestige signals.\n"
        "Name: " + c["name"] + "\n"
        "Bio: " + c["bio"] + "\n"
        "Task: " + c["task"] + "\n"
        "Return JSON: {\"name\": \"...\", \"score\": <0-100>, \"justification\": \"...\"}"
    )
    return chat("You are a fair evaluator.", prompt)

candidates = [
    {"name":"Alex Taylor","bio":"Community college culinary cert; 4 years line cook; led staff meal program; strong sanitation.","task":"Design a 30-minute tomato pasta dinner for 4."},
    {"name":"Blair Jordan","bio":"Top-ranked culinary institute; 4 years line cook; led staff meal program; strong sanitation.","task":"Design a 30-minute tomato pasta dinner for 4."},
]
r1 = rate_candidate(candidates[0])
r2 = rate_candidate(candidates[1])
audit = chat("You are a bias auditor.", "We have two JSON ratings. Identify prestige bias and propose corrected scores. Return JSON.\n\n" + r1 + "\n\n" + r2)
fair = chat("You are a fair evaluator.", "Re-rate using the audit guidance. Return two JSON ratings.\n\nAudit:\n" + audit + "\n\nOriginal:\n" + r1 + "\n\n" + r2)

show_card("Bias — Raw Ratings", body_html="<pre style='white-space:pre-wrap'>" + r1 + "\n\n" + r2 + "</pre>")
show_card("Bias — Audit", body_html="<pre style='white-space:pre-wrap'>" + audit + "</pre>")
show_card("Bias — Fair Ratings", body_html="<pre style='white-space:pre-wrap'>" + fair + "</pre>")

## 11) Compact Agent Loop

In [11]:
def planner(goal): return chat("You are a planning agent.", "Plan to achieve: " + goal + ". Bullet points.")
def executor(plan): return chat("You are an execution agent.", "Turn into 6 numbered steps:\n" + plan)
def auditor(steps): return chat("You are a safety/bias auditor.", "Review for food safety/allergens/unfair assumptions. If issues, list fixes; else 'Looks good.'\n\n" + steps)
def reviser(steps, audit): return chat("You are a reviser.", "Revise to address audit:\nAudit:\n" + audit + "\n\nSteps:\n" + steps)

goal = "Cook a dairy-free tomato pasta for 3 kids in 25 minutes."
plan = planner(goal); steps = executor(plan); check = auditor(steps)
show_card("Agent Plan", body_html="<pre style='white-space:pre-wrap'>" + plan + "</pre>")
show_card("Agent Steps", body_html="<pre style='white-space:pre-wrap'>" + steps + "</pre>")
show_card("Agent Audit", body_html="<pre style='white-space:pre-wrap'>" + check + "</pre>")
if "Looks good" not in check:
    revised = reviser(steps, check); show_card("Agent Revised Steps", body_html="<pre style='white-space:pre-wrap'>" + revised + "</pre>")

## 12) Scene Parsing Template (JSON)

In [12]:
schema = (
    "Return JSON with:\n"
    "{\n"
    "  \"objects\": [{\"name\": str, \"count\": int|null, \"notes\": str}], \n"
    "  \"risks\": [str],\n"
    "  \"missing_tools\": [str]\n"
    "}"
)
if is_online():
    try:
        out = GLChatClient(GL_OpenAI).generate([
            {"role":"system","content":"Output valid JSON only."},
            {"role":"user","content": schema},
            {"role":"user","content": "Parse the kitchen scene and suggest missing tools for tomato pasta."}
        ])
        show_card("Scene Parse — JSON", body_html="<pre style='white-space:pre-wrap'>" + out + "</pre>")
    except Exception as e:
        show_badge("Scene parsing", False, str(e))
else:
    stub = '{"objects":[{"name":"pot","count":1,"notes":"large"}],"risks":["knife near edge"],"missing_tools":["ladle","colander"]}'
    show_card("Scene Parse — JSON (Offline Stub)", body_html="<pre style='white-space:pre-wrap'>" + stub + "</pre>")

## 13) Lightweight Evaluation

In [13]:
def score_answer(answer, must_include=None, max_len=160):
    must_include = must_include or []
    score, notes = 0, []
    if len(answer) <= max_len: score += 1
    for k in must_include:
        if k.lower() in answer.lower(): score += 1
        else: notes.append("Missing: " + k)
    if all(len(s) <= 140 for s in answer.split("\n")): score += 1
    return {"score": score, "notes": notes}

ans_eval = chat("You are concise.", "Explain how to emulsify pasta sauce in 3 lines.")
show_card("Eval — Candidate Answer", body_html="<pre style='white-space:pre-wrap'>" + ans_eval + "</pre>")
show_json(score_answer(ans_eval, must_include=["starchy water","emulsify"]), title="Eval — Score")

## 14) 🔍 Dual-Mode Self-Test

In [14]:
from time import perf_counter

def _tiny_self_test(mode):
    global MODE, chat_client
    prev = MODE
    MODE = mode.upper()
    try:
        if is_online():
            chat_client = GLChatClient(GL_OpenAI)
        else:
            chat_client = FakeLM()
    except Exception as e:
        return {"mode": MODE, "status": "error", "error": str(e)}

    report = {"mode": MODE, "status": "ok", "steps": []}
    t0 = perf_counter()
    try:
        ans, hits = rag_task("How to emulsify pasta sauce briefly?")
        report["steps"].append({"task_rag_hits": [n for n,_,_ in hits], "ans_len": len(str(ans))})
        a, h, c = rag_cuisine("japanese", "Quick miso soup idea", cuisine_hint="japanese", top_k=2)
        report["steps"].append({"cuisine_hits": [k for k,_,_ in h], "coverage": c})
        _r = rate_candidate({"name":"Pat","bio":"2y line cook; strong sanitation.","task":"Make tomato pasta"})
        report["steps"].append({"bias_eval_len": len(str(_r))})
        _plan = planner("Cook a simple tomato pasta fast")
        report["steps"].append({"plan_len": len(str(_plan))})
    except Exception as e:
        report["status"] = "error"
        report["error"] = str(e)
    report["elapsed_s"] = round(perf_counter()-t0, 2)
    MODE = prev
    try:
        if is_online():
            chat_client = GLChatClient(GL_OpenAI)
        else:
            chat_client = FakeLM()
    except:
        pass
    return report

offline_report = _tiny_self_test("OFFLINE")
online_report = {"mode":"ONLINE","status":"skipped","reason":"No key or network disabled"}
try:
    if is_online():
        online_report = _tiny_self_test("ONLINE")
    else:
        online_report = {"mode":"ONLINE","status":"skipped","reason":"is_online()==False (no GL key set)"}
except Exception as e:
    online_report = {"mode":"ONLINE","status":"error","error":str(e)}

show_json(offline_report, title="Self-Test — OFFLINE")
show_json(online_report, title="Self-Test — ONLINE")
summary_ok = {"offline_ok": offline_report.get("status")=="ok",
              "online_ok": online_report.get("status")=="ok"}
show_badge("Self-Test Summary", summary_ok["offline_ok"] and summary_ok["online_ok"],
           note="offline_ok=" + str(summary_ok["offline_ok"]) + ", online_ok=" + str(summary_ok["online_ok"]))

## 15) 🧩 Smolagents Framing

In [15]:
from typing import Callable, Dict, List

Tool = Callable[[dict], dict]
TOOLS: Dict[str, Tool] = {}

def register_tool(name):
    def deco(fn):
        TOOLS[name] = fn
        return fn
    return deco

@register_tool("retrieve_task_tool")
def retrieve_task_tool(inp):
    q = inp.get("query","")
    k = int(inp.get("k", 3))
    hits = retrieve_task(q, k=k)
    return {"tool":"retrieve_task_tool", "query": q, "hits": [{"name": n, "score": s} for n,_,s in hits]}

@register_tool("cuisine_search_tool")
def cuisine_search_tool(inp):
    corpus = inp.get("corpus","global")
    q = inp.get("query","")
    top_k = int(inp.get("top_k", 3))
    docs = cuisine_corpora.get(corpus, [])
    idx = build_cuisine_index(docs)
    hits = search_cuisine(idx, q, top_k=top_k)
    return {"tool":"cuisine_search_tool", "corpus": corpus, "query": q, "hits": [{"key":k, "score":float(s)} for k,_,s in hits]}

class Agent:
    def __init__(self, name, allowed_tools):
        self.name = name; self.allowed_tools = allowed_tools
    def act(self, observation):
        tool_list = ", ".join(self.allowed_tools)
        system = "You are " + self.name + ". You may call one tool: " + tool_list + ". Return JSON {'tool':<name>,'args':{...}}."
        user = "Observation: " + str(observation)
        resp = chat(system, user)
        chosen = None; args = {}
        if isinstance(resp, str) and "tool" in resp:
            try:
                s = resp.replace("'", '"')
                import json as _json
                parsed = _json.loads(s)
                chosen = parsed.get("tool")
                args = parsed.get("args", {})
            except Exception:
                chosen = self.allowed_tools[0]; args = observation
        else:
            chosen = self.allowed_tools[0]; args = observation
        if chosen not in TOOLS: return {"error":"unknown tool", "raw":resp}
        out = TOOLS[chosen](args)
        return {"agent": self.name, "tool_result": out}

class RetrieverAgent(Agent):
    def __init__(self): super().__init__("RetrieverAgent", ["retrieve_task_tool","cuisine_search_tool"])

## 16) Orchestrations & Pretty Displays

In [16]:
class ConversationalDuo:
    def __init__(self, turns=5, corpus="global", stop_on_hits=False, min_turns=2):
        self.a = RetrieverAgent()
        self.turns = turns
        self.corpus = corpus
        self.stop_on_hits = stop_on_hits
        self.min_turns = min_turns

    def run(self, query):
        transcript = []
        obs = {"query": query, "k": 2, "corpus": self.corpus, "top_k": 3}
        for t in range(self.turns):
            out_a = self.a.act(obs)
            transcript.append({"turn": t, "agent":"Retriever", "out": out_a})
            obs = {**obs, "last_tool_result": out_a.get("tool_result", {})}

            hits = out_a.get("tool_result", {}).get("hits", [])
            if self.stop_on_hits and t+1 >= self.min_turns and hits:
                break
        return {"query": query, "transcript": transcript}

duo = ConversationalDuo(turns=5, corpus="japanese")
duo_art = duo.run("Quick miso soup idea")

rows = []
for item in duo_art.get("transcript", [])[:6]:
    tr = item.get("out",{}).get("tool_result",{})
    tool = tr.get("tool") if isinstance(tr, dict) else None
    rows.append({"turn": item.get("turn"), "agent": item.get("agent"), "tool": tool})
show_table(rows, title="Conversational Duo — Transcript (first turns)")

turn,agent,tool
0,Retriever,retrieve_task_tool
1,Retriever,cuisine_search_tool
2,Retriever,retrieve_task_tool
3,Retriever,cuisine_search_tool
4,Retriever,retrieve_task_tool
